# COVID-19 Data Analysis
This notebook is for playing around with the `Johns Hopkins University` COVID-19 Data.

It uses the `covid` module located alongside it.

## 1.1 Methods

The data is collected in `pandas.DataFrame`s:
* `zone_df` is pandas DataFrame containing information on each zone.
* `ts_df` is a pandas DataFrame containing time-series COVID metrics

World Bank Indicators are used through the `wbdata` interface in order to find correlations and the population size of each country. Country names are normalised to fit the `JHU` names.

`pycountry` and `pycountry_convert` is used to lookup which continent a given country is located in to enable easy data slicing later on.

A logistic fit is made using `lmfit.Model` and plotted.

Correlations are then found and explored by calculating the Pearson correlation coefficient for the data stored in `zone_df`.


In [2]:
## Initialisation
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import geopandas
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pycountry
import pycountry_convert
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import bokeh
import bokeh.palettes
import plotly.express as px
import plotly.graph_objects as go
import wbdata
from covid.grabbers import grab_wbdata, grab_JHU
from covid.utils import (print_wb_indicators, print_wb_sources, get_x_day, rchop, drop_y,
                         get_latest_valid, country_to_continent)
from covid.constants import WB_DF_RENAMED_COUNTRIES
from covid.statistics import LogisticModel


## 2.1 Setting up `zone_df`

In [3]:
### Add stats to zone_df
def process_wb_indicator(indicator, column_name=None):
    if column_name is None:
        column_name = indicator
    
    df = grab_wbdata({indicator: column_name})
    df = get_latest_valid(df).reset_index(level=1, drop=True)

    df.index.name='zone'
    df.rename(index=WB_DF_RENAMED_COUNTRIES, inplace=True)

    return df

zone_df = pd.DataFrame(columns=['zone'])

# Subnational population. Source=50
zone_df = pd.merge(zone_df, process_wb_indicator('SP.POP.TOTL', 'population'), on='zone', how='outer', suffixes=('', '_y'))

# Degree of urbanisation
zone_df = pd.merge(zone_df, process_wb_indicator('SP.URB.TOTL.IN.ZS', 'urbanisation'), on='zone', how='outer', suffixes=('', '_y'))

# Hospital beds per 1K population
zone_df = pd.merge(zone_df, process_wb_indicator('SH.MED.BEDS.ZS', 'hospital_beds_per_1K'), on='zone', how='outer', suffixes=('', '_y'))

# 
# grab indicators and load into data frame
wb_df = grab_wbdata({'VA.EST': 'voice_and_accountability',
                     'GE.EST': 'government_effectiveness', })

gov_df = wb_df.groupby(level='country').tail(5).groupby(level='country').mean()
gov_df.index.name = 'zone'
gov_df.rename(index=WB_DF_RENAMED_COUNTRIES, inplace=True)

# Merge into zone_df
zone_df = pd.merge(zone_df, gov_df, on='zone', how='outer', suffixes=('', '_y'))
drop_y(zone_df)

# Add continents to zone_df
zone_df['continent'] = zone_df['zone'].apply(country_to_continent)

del wb_df
del gov_df
zone_df

Failed to process Arab World due to error: arab world
Failed to process Caribbean small states due to error: caribbean small states
Failed to process Central Europe and the Baltics due to error: central europe and the baltics
Failed to process Channel Islands due to error: channel islands
Failed to process Congo (Kinshasa) due to error: congo (kinshasa)
Failed to process Congo (Brazzaville) due to error: congo (brazzaville)
Failed to process Early-demographic dividend due to error: early-demographic dividend
Failed to process East Asia & Pacific due to error: east asia & pacific
Failed to process East Asia & Pacific (IDA & IBRD countries) due to error: east asia & pacific (ida & ibrd countries)
Failed to process East Asia & Pacific (excluding high income) due to error: east asia & pacific (excluding high income)
Failed to process Euro area due to error: euro area
Failed to process Europe & Central Asia due to error: europe & central asia
Failed to process Europe & Central Asia (IDA & I

,zone,population,urbanisation,hospital_beds_per_1K,voice_and_accountability,government_effectiveness,continent
0,Afghanistan,37172386.0,25.495,0.5,-1.718050,-1.838644,Asia
1,Albania,2866376.0,60.319,2.9,-0.251730,-0.629331,Europe
2,Algeria,42228429.0,72.629,1.9,-1.128412,-0.818300,Africa
3,American Samoa,55465.0,87.153,NaN,NaN,NaN,Oceania
4,Andorra,77006.0,88.062,2.5,1.497404,1.431379,Europe
...,...,...,...,...,...,...,...
321,Martinique,NaN,NaN,NaN,0.719746,0.909367,North America
322,Netherlands Antilles,NaN,NaN,NaN,NaN,NaN,Unknown
323,Niue,NaN,NaN,NaN,NaN,NaN,Oceania
324,Reunion,NaN,NaN,NaN,NaN,NaN,Africa


## 2.2 Setup parameters

In [19]:
MIN_REGRESSION_DATAPOINTS = 5

label = 'deaths_per_1M'
regression_predict_length = 10
regression_confidence_levels = 1  # 0 to disable
fit_method = 'leastsq'
log_scale = True
annotate_country = True
# countries = 'all'
# countries = ['United Kingdom', 'Italy', 'Spain', 'Denmark', 'Iran', 'Greece', 'US', 'Sweden', 'Austria', 'Germany', 'Norway', 'Japan', 'Australia', 'France', 'China']
countries = ['Denmark', 'US', 'Iran', 'United Kingdom', 'Sweden', 'Italy', 'Spain']

if label.startswith('cases'):
    n = 100
else:
    n = 10



## 2.3 Setting up `ts_df`

In [20]:
## Timeseries
# Grab JHU time series data

ts_df = grab_JHU()

if countries == 'all':
    countries = ts_df.reset_index()['country'].unique()

# Add population to ts_df
ts_df = pd.merge(ts_df.reset_index(), zone_df.set_index('zone')['population'], left_on='country', right_on='zone', how='outer')
ts_df.set_index(['country', 'date'], inplace=True)
ts_df.sort_index(inplace=True)

# Calculate per population
for metric in ('cases', 'deaths', 'recoveries'):
    ts_df[metric + '_per_1M'] = (ts_df[metric] * 10**6) / ts_df['population']

# Calculate outcome
ts_df['deaths_per_recovery'] = ts_df['deaths'] / ts_df['recoveries']

# Calculate x day
x_days = ts_df.groupby('country').apply(lambda x: get_x_day(x, rchop(label, '_per_1M'), n))
ts_df = pd.merge(ts_df, x_days.rename('x_day'), on='country', suffixes=('', '_y'))
drop_y(ts_df)

ts_df['rel_day'] = ts_df['day'] - ts_df['x_day']


In [76]:
import matplotlib.colors
str(matplotlib.colors.to_rgba('#ffffff05'))

'(1.0, 1.0, 1.0, 0.0196078431372549)'

## 2.4 Run regression and plot

In [80]:
fig = go.Figure()

if log_scale:
    #ax.set_yscale('log')
    fig.update_yaxes(type='log')

# Calculate color scale
colors = {i: x for i, x in enumerate(bokeh.palettes.viridis(len(countries) + 1))}

countries_fit = []
for i, country in enumerate(countries):
    print(f'Processing country: {country}')

    sub_df = ts_df.loc[country][ts_df.loc[country]['province_state'] == 'total']
    rel_df = sub_df.query('rel_day >= 1')

    if len(rel_df) < MIN_REGRESSION_DATAPOINTS:
        continue

    model = LogisticModel
    params = model.make_params(a=1000, b=0.3, c=1000)
    params['a'].min = 0
    params['b'].min = 0
    params['b'].max = 0.5
    params['c'].min = 0
    try:
        result = model.fit(rel_df[label], params, x=rel_df['rel_day'], method=fit_method)
    except (ValueError, TypeError) as e:
        print(f'Skipping {country}. Got fit error: {e}')

    countries_fit.append({
        'zone': country,
        'fit_result': result,
        'fit_a': result.params['a'].value,
        'fit_b': result.params['b'].value,
        'fit_c': result.params['c'].value,
        'cases': rel_df.iloc[-1]['cases'],
        'deaths': rel_df.iloc[-1]['deaths'],
        'recoveries': rel_df.iloc[-1]['recoveries'],
        'cases_per_1M': rel_df.iloc[-1]['cases_per_1M'],
        'deaths_per_1M': rel_df.iloc[-1]['deaths_per_1M'],
        'recoveries_per_1M': rel_df.iloc[-1]['recoveries_per_1M'],
        'deaths_per_recovery': rel_df.iloc[-1]['deaths_per_recovery'],
        'lat': rel_df.iloc[-1]['lat'],
        'long': rel_df.iloc[-1]['long'],
        'day': rel_df.iloc[-1]['day'],
        'rel_day': rel_df.iloc[-1]['rel_day'],
        'x': rel_df['rel_day'],
        'y': rel_df[label],
        'n': n,
        'col': colors[i]
    })

    # Labels with result parameters
    axlbl = f'a: {result.params["a"].value:.0f}\nb: {result.params["b"].value:.3f}\nc: {result.params["c"].value:.5f}'.replace('\n', '<br>')

    fig.add_trace(go.Scattergl(x=rel_df['rel_day'], y=rel_df[label], name=country, legendgroup=country, mode='markers', marker=dict(color=colors[i]), hovertemplate=f'<b>{country}</b><br>' + '(%{x}, %{y})<br><extra></extra>'))
   
    # Regression 
    last_rel_day = rel_df['rel_day'].iloc[-1]
    regression_predict_days = np.arange(last_rel_day + 1, last_rel_day + regression_predict_length + 1)

    regression_range = np.append(rel_df['rel_day'].values, regression_predict_days)

    fig.add_trace(go.Scattergl(x=regression_range, y=result.eval(x=regression_range), name=country, legendgroup=country, mode='lines',
                            line=dict(color=colors[i]), showlegend=False,
                            hovertemplate=f'<b>{country}</b><br>' + '(%{x}, %{y})<br>' + axlbl + '<extra></extra>'))

    # Regression confidence intervals
    for j in range(1, regression_confidence_levels + 1):
        p = result.params
        fig.add_trace(go.Scatter(x=np.concatenate([regression_range, regression_range[::-1]]), y=np.concatenate([model.func(regression_range, p['a'] + j * p['a'].stderr, p['b'] + j * p['b'].stderr, p['c'] + j * p['c'].stderr), model.func(regression_range, p['a'] - j * p['a'].stderr, p['b'] - j * p['b'].stderr, p['c'] - j * p['c'].stderr)[::-1]]), showlegend=False, legendgroup=country, hoverinfo='skip', mode='lines', fill='toself', line_color='rgba(255, 255, 255, 0)', fillcolor='rgba'+str(matplotlib.colors.to_rgba(colors[i]+'55'))))

fig.update_layout(
    title=f'COVID-19 related {rchop(label, "_per_1M")} for a range of countries',
    xaxis_title=f'Days since {n} {rchop(label, "_per_1M")}',
    yaxis_title=' '.join(label.split('_')).capitalize(),
    hovermode='closest',
    hoverdistance=-1
)

# Update zone_df
countries_df = pd.DataFrame(countries_fit)
zone_df = pd.merge(zone_df, countries_df, on='zone', suffixes=('', '_y'))
drop_y(zone_df)

fig.show()

Processing country: Denmark
Processing country: US
Processing country: Iran
Processing country: United Kingdom
Processing country: Sweden
Processing country: Italy
Processing country: Spain


In [46]:
    np.concatenateregression_range, regression_range)


TypeError: only integer scalar arrays can be converted to a scalar index

## 2.5 Find correlations

In [81]:
covid_metric = 'deaths_per_1M'
other_metric = 'urbanisation'

subdf = zone_df
subdf = subdf.query('deaths > 25 & continent == "Europe"')

#zone_df.plot.scatter(x='urbanisation', y=metric, ax=ax, ylim=(0, 10000))
#subdf[['urbanisation', metric]].corr()
# px.scatter(x=zone_df['urbanisation'], y=zone_df[metric])
#tls.mpl_to_plotly(fig)
lin_model = ols(f'{covid_metric} ~ {other_metric}', data=subdf).fit()
plot = px.scatter(subdf, x=other_metric, y=covid_metric, hover_name='zone')
plot.show()

corr = subdf.corr()
#print(corr)
print(corr.loc[other_metric][covid_metric])
corr


-0.8238893068386798


,population,urbanisation,hospital_beds_per_1K,voice_and_accountability,government_effectiveness,fit_a,fit_b,fit_c,cases,deaths,recoveries,cases_per_1M,deaths_per_1M,recoveries_per_1M,deaths_per_recovery,lat,long,day,rel_day,n
population,1.000000,-0.713761,0.752351,-0.882585,-0.575263,0.915670,-0.095612,0.762669,0.787178,0.838154,0.401968,0.478929,0.682206,0.187643,0.521480,-0.701903,-0.625233,NaN,0.729670,NaN
urbanisation,-0.713761,1.000000,-0.991102,0.957868,0.969043,-0.416906,0.726256,-0.796550,-0.838880,-0.909822,-0.601740,-0.656241,-0.823889,-0.396353,0.156566,0.039176,-0.084640,NaN,-0.993808,NaN
hospital_beds_per_1K,0.752351,-0.991102,1.000000,-0.963191,-0.932534,0.469148,-0.643451,0.867696,0.890390,0.946837,0.663456,0.716748,0.880461,0.446844,-0.127620,-0.070374,0.042684,NaN,0.999082,NaN
voice_and_accountability,-0.882585,0.957868,-0.963191,1.000000,0.884319,-0.645628,0.534996,-0.825201,-0.872981,-0.943388,-0.553889,-0.622367,-0.815413,-0.336118,-0.110320,0.318945,0.204137,NaN,-0.956684,NaN
government_effectiveness,-0.575263,0.969043,-0.932534,0.884319,1.000000,-0.281539,0.858668,-0.634095,-0.682121,-0.779458,-0.444686,-0.496443,-0.675973,-0.248718,0.231229,-0.075740,-0.218662,NaN,-0.940398,NaN
fit_a,0.915670,-0.416906,0.469148,-0.645628,-0.281539,1.000000,0.213970,0.542922,0.517676,0.570819,0.097551,0.173793,0.398428,-0.117025,0.799339,-0.883181,-0.827055,NaN,0.434665,NaN
fit_b,-0.095612,0.726256,-0.643451,0.534996,0.858668,0.213970,1.000000,-0.234870,-0.311457,-0.407787,-0.234030,-0.254049,-0.346219,-0.149109,0.555806,-0.464044,-0.593393,NaN,-0.668961,NaN
fit_c,0.762669,-0.796550,0.867696,-0.825201,-0.634095,0.542922,-0.234870,1.000000,0.971986,0.952281,0.852610,0.888887,0.978837,0.660332,-0.051900,-0.153177,-0.109576,NaN,0.857166,NaN
cases,0.787178,-0.838880,0.890390,-0.872981,-0.682121,0.517676,-0.311457,0.971986,1.000000,0.985063,0.871661,0.911461,0.983623,0.714927,-0.074399,-0.176087,-0.129856,NaN,0.885724,NaN
deaths,0.838154,-0.909822,0.946837,-0.943388,-0.779458,0.570819,-0.407787,0.952281,0.985063,1.000000,0.788610,0.839144,0.956761,0.603413,-0.023020,-0.217852,-0.145145,NaN,0.941508,NaN


In [8]:
fig = plt.figure(figsize=(15, 8))
fig = sm.graphics.plot_regress_exog(lin_model, other_metric, fig=fig)
lin_model.summary2()

NameError: name 'lin_model' is not defined

In [9]:

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

world.rename(columns={'name': 'zone'}, inplace=True)

a = world.merge(new_df, on='zone', how='outer')

a.query('continent == "Europe"').query('zone != "Russia"').plot(column='deaths_per_1M', legend=True, cmap='summer', missing_kwds={
    "color": 'lightgrey'
})

NameError: name 'new_df' is not defined

# Country Bayersian fit


import lmfit
country = 'Denmark'
province_state = 'total'
label = 'cases'
regression_predict_length = 50
log_scale = True
emcee_factor = 1.0/1

c_df = ts_df.loc[country].query(f'province_state == "{province_state}"')

rel_df = c_df.query('rel_day >= 1')
# rel_df.plot(x='rel_day', y=label)

model = LogisticModel
params = model.make_params(a=1000, b=0.3, c=1000)
params['a'].min = 0
params['b'].min = 0
params['b'].max = 0.5
params['c'].min = 0
result_leastsq = model.fit(rel_df[label], params, x=rel_df['rel_day'], method='leastsq', nan_policy='omit')
result_nelder = model.fit(rel_df[label], params, x=rel_df['rel_day'], method='Nelder', nan_policy='omit')

emcee_kws = {
    'steps': int(round(1000 * emcee_factor)),
    'burn': int(round(300 * emcee_factor)),
#    'thin': 20,
    'is_weighted': False,
    'progress': True
}

emcee_params = result_nelder.params.copy()
emcee_params.add('__lnsigma', value=np.log(0.1), min=np.log(0.001), max=np.log(2.0))

result_emcee = model.fit(rel_df[label], emcee_params, x=rel_df['rel_day'], method='emcee', nan_policy='omit', fit_kws=emcee_kws)

last_rel_day = rel_df['rel_day'].iloc[-1]
regression_predict_days = np.arange(last_rel_day + 1, last_rel_day + regression_predict_length + 1)

regression_range = np.append(rel_df['rel_day'].values, regression_predict_days)

# result.plot()

fig, ax = plt.subplots(figsize=(20, 6))
if log_scale:
    ax.set_yscale('log')

p = result_leastsq.params

plt.scatter(rel_df['rel_day'], rel_df[label])
plt.plot(regression_range, model.func(regression_range, p['a'], p['b'], p['c']))
# Plot confidence intervals
for i in range(1, 2):
    dely = result_leastsq.eval_uncertainty(sigma=i, x=regression_range)
    plt.fill_between(regression_range, model.func(regression_range, p['a'], p['b'], p['c']) + dely, model.func(regression_range, p['a'], p['b'], p['c']) - dely, color='#ABABAB', alpha=0.3, label=f'{i} $\sigma$ confidence interval')

plt.legend()
#e_result = result.emcee()

In [10]:
fig, ax = plt.subplots(figsize=(20, 6))
#ax.set_yscale('log')
ax.set_ylabel(label)
sigma_n = 1

# Emcee fit
p = result_emcee.params
color = (1.0, 0.0, 0.0)
#plt.fill_between(regression_range, model.func(regression_range, p['a'], p['b'] + p['b'].stderr, p['c']), model.func(regression_range, p['a'], p['b'] - p['b'].stderr, p['c']), color=color, alpha=0.2)
#plt.plot(regression_range, model.func(regression_range, p['a'], p['b'], p['c']), color=color, label='emcee fit')

# Least squares fit
p = result_leastsq.params
color = (0.0, 1.0, 0.0)
#plt.fill_between(regression_range, model.func(regression_range, p['a'] + p['a'].stderr, p['b'] + p['b'].stderr, p['c'] + p['c'].stderr), model.func(regression_range, p['a'] - p['a'].stderr, p['b'] - p['b'].stderr, p['c'] - p['c'].stderr), color=color, alpha=0.2)
#plt.plot(regression_range, model.func(regression_range, p['a'], p['b'], p['c']), color=color, label='leastsq fit')
plt.fill_between(regression_range, model.func(regression_range, p['a'] + sigma_n * p['a'].stderr, p['b'] + sigma_n * p['b'].stderr, p['c'] + sigma_n * p['c'].stderr), model.func(regression_range, p['a'] - sigma_n * p['a'].stderr, p['b'] - sigma_n * p['b'].stderr, p['c'] - sigma_n * p['c'].stderr), color=color, alpha=0.2)

# Nelder fit
p = result_nelder.params
color = (0.0, 0.0, 1.0)
#plt.fill_between(regression_range, model.func(regression_range, p['a'], p['b'] + p['b'].stderr, p['c']), model.func(regression_range, p['a'], p['b'] - p['b'].stderr, p['c']), alpha=0.2)
#plt.plot(regression_range, model.func(regression_range, p['a'], p['b'], p['c']), color=color, label='Nelder fit')

# Test
p = result_emcee.params
color = (0.0, 0.0, 0.0)
#plt.fill_between(regression_range, model.func(regression_range, p['a'] + sigma_n * p['a'].stderr, p['b'] + sigma_n * p['b'].stderr, p['c'] + sigma_n * p['c'].stderr), model.func(regression_range, p['a'] - sigma_n * p['a'].stderr, p['b'] - sigma_n * p['b'].stderr, p['c'] - sigma_n * p['c'].stderr), color=color, alpha=0.2)
#plt.plot(regression_range, model.func(regression_range, p['a'], p['b'], p['c']), color=color, label='test fit')


# Actual data
plt.scatter(rel_df['rel_day'], rel_df[label], label=f'{country}')
plt.legend()

NameError: name 'result_emcee' is not defined